### Limpeza e preparação dos dados

---


### Tratando dados ausentes
Dados ausentes são comuns em muitas aplicações de análise de
dados. Um dos objetivos do pandas é deixar o trabalho com dados
ausentes o menos problemático possível. Por exemplo, todas as
estatísticas descritivas em objetos do pandas, por padrão, excluem
dados ausentes.

A forma como os dados ausentes são representados em objetos do
pandas, de certo modo, não é perfeita, porém é funcional para
muitos usuários. Para dados numéricos, o pandas utiliza o valor de
ponto flutuante NaN (Not a Number) para representá-los. Esse valor
é chamado de *valor de sentinela*, e pode ser facilmente detectado:

In [95]:
import numpy as np
import pandas as pd

In [96]:
string_data = pd.Series(['aardvark','artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [97]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

No pandas, adotamos uma convenção usada na linguagem de
programação R, referenciando os dados ausentes como NA, que
quer dizer *Not Available* (indisponível). Em aplicações estatísticas,
dados NA podem ser dados inexistentes ou dados que existem,
porém não foram observados (por causa de problemas com a coletade dados, por exemplo). Ao limpar os dados para análise, em geral
é importante fazer a análise nos próprios dados ausentes a fim de
identificar problemas em sua coleta ou possíveis distorções
provocadas por dados ausentes.

O valor embutido None de Python *também é tratado como* NA em arrays
de objetos:

In [98]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

Há um trabalho em andamento no projeto pandas cujo objetivo é
melhorar os detalhes internos do tratamento de dados ausentes,
mas as funções de API dos usuários, como pandas.isnull , abstraem
muitos dos detalhes incômodos. Veja a Tabela abaixo que apresenta
uma lista de algumas funções relacionadas ao tratamento de dados
ausentes.

### Tabela – Métodos para tratamento de NA

Argumento | Descrição
----------|-----------
dropna | Filtra rótulos de eixos, baseado no fato de os valores para cada
|rótulo terem dados ausentes, com limites variados para a
|quantidade de dados ausentes a ser tolerada.
fillna | Preenche os dados ausentes com algum valor ou utilizando um
|método de interpolação como 'ffill' ou 'bfill'.
isnull | Devolve valores booleanos informando quais valores estão
|ausentes/são NA.
notnull | Negação de isnull.


### Filtrando dados ausentes
Há algumas maneiras de filtrar dados ausentes. Embora semprehaja a opção de fazer isso manualmente usando *pandas.isnull* e uma
indexação booleana, o método dropna pode ser útil. Em uma Series,
ele devolve a Series somente com os dados diferentes de null e os
valores dos índices:

In [99]:
from numpy import nan as NA 

In [100]:
data = pd.Series([1, NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [101]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

Essa instrução é equivalente a:

In [102]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

Com objetos DataFrame, a situação é um pouco mais complexa.
Talvez você queira descartar linhas ou colunas que contenham
somente NA ou apenas aquelas que contenham algum NA. Por
padrão, *dropna* descarta qualquer linha contendo um valor ausente:

In [103]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [104]:
data.dropna()

,0,1,2
0,1.0,6.5,3.0


Passar *how='all'* descartará apenas as linhas que contenham
somente NAs:

In [105]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


Para descartar colunas do mesmo modo, passe axis=1 :

In [106]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [107]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


Um modo relacionado de filtrar linhas de DataFrame diz respeito a
dados de séries temporais. Suponha que queremos manter somente
as linhas contendo determinado número de observações. Podemos
representar isso com o argumento *thresh*:

In [108]:
df = pd.DataFrame(np.random.randn(7, 3))
df

,0,1,2
0,1.007189,-1.296221,0.274992
1,0.228913,1.352917,0.886429
2,-2.001637,-0.371843,1.669025
3,-0.438570,-0.539741,0.476985
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


In [109]:
#      [linha, coluna]
df.iloc[:4, 1] = NA

In [110]:
df

,0,1,2
0,1.007189,NaN,0.274992
1,0.228913,NaN,0.886429
2,-2.001637,NaN,1.669025
3,-0.438570,NaN,0.476985
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


In [111]:
df.iloc[:2, 2] = NA

In [112]:
df

,0,1,2
0,1.007189,NaN,NaN
1,0.228913,NaN,NaN
2,-2.001637,NaN,1.669025
3,-0.438570,NaN,0.476985
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


In [113]:
df.dropna()

,0,1,2
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


In [114]:
df.dropna(thresh=2)

,0,1,2
2,-2.001637,NaN,1.669025
3,-0.438570,NaN,0.476985
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


### Preenchendo dados ausentes
Em vez de filtrar dados ausentes (e possivelmente descartar outros
dados junto com esses), você poderá preencher as “lacunas” de
várias maneiras. Na maioria dos casos, o método *fillna* será a função
que representa a força de trabalho a ser utilizada. Chamar *fillna* com
uma constante substitui valores ausentes por esse valor:

In [115]:
df.fillna(0)

,0,1,2
0,1.007189,0.000000,0.000000
1,0.228913,0.000000,0.000000
2,-2.001637,0.000000,1.669025
3,-0.438570,0.000000,0.476985
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


Ao chamar *fillna* com um dicionário, podemos usar um valor de
preenchimento diferente para cada coluna:

In [116]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,1.007189,0.500000,0.000000
1,0.228913,0.500000,0.000000
2,-2.001637,0.500000,1.669025
3,-0.438570,0.500000,0.476985
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


*fillna* devolve um novo objeto, mas o objeto existente pode ser
alterado in-place:

In [117]:
_ = df.fillna(0, inplace=True)

In [118]:
df

,0,1,2
0,1.007189,0.000000,0.000000
1,0.228913,0.000000,0.000000
2,-2.001637,0.000000,1.669025
3,-0.438570,0.000000,0.476985
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


Os mesmos métodos de interpolação disponíveis para reindexação
podem ser usados com *fillna*:

In [119]:
df = pd.DataFrame(np.random.randn(6, 3))
df

,0,1,2
0,-0.831154,-2.370232,-1.860761
1,-0.860757,0.560145,-1.265934
2,0.119827,-1.063512,0.332883
3,-2.359419,-0.199543,-1.541996
4,-0.970736,-1.307030,0.286350
5,0.377984,-0.753887,0.331286


In [120]:
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,-0.831154,-2.370232,-1.860761
1,-0.860757,0.560145,-1.265934
2,0.119827,NaN,0.332883
3,-2.359419,NaN,-1.541996
4,-0.970736,NaN,NaN
5,0.377984,NaN,NaN


In [121]:
df.fillna(method='ffill')

,0,1,2
0,-0.831154,-2.370232,-1.860761
1,-0.860757,0.560145,-1.265934
2,0.119827,0.560145,0.332883
3,-2.359419,0.560145,-1.541996
4,-0.970736,0.560145,-1.541996
5,0.377984,0.560145,-1.541996


In [122]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,-0.831154,-2.370232,-1.860761
1,-0.860757,0.560145,-1.265934
2,0.119827,0.560145,0.332883
3,-2.359419,0.560145,-1.541996
4,-0.970736,NaN,-1.541996
5,0.377984,NaN,-1.541996


Com *fillna*, podemos executar várias outras tarefas com um pouco de
criatividade. Por exemplo, podemos passar o valor da média ou da
mediana de uma Series:

In [123]:
data = pd.Series([1., NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [124]:
media = data.mean()
media

3.8333333333333335

In [125]:
data.fillna(media)

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

### Tabela – Argumentos da função fillna
Argumento | Descrição
----------|-----------
value | Valor escalar ou um objeto do tipo dicionário a ser usado para
|preencher valores ausentes
method | Interpolação; por padrão, será 'ffill' se a função for chamada 
|sem outros argumentos
axis | Eixo a ser preenchido; o default é axis=0
inplace | Modifica o objeto que faz a chamada, sem gerar uma cópia
limit | Para preenchimento para a frente (forward) e para trás (backward),
| é o número máximo de valores consecutivos a serem preenchidos

### Transformação de dados
Até agora neste capítulo, estivemos preocupados com a
reorganização dos dados. Filtragem, limpeza e outras
transformações constituem outra classe de operações importantes.

### Removendo duplicatas

Linhas duplicadas podem ser encontradas em um DataFrame por
diversos motivos. Eis um exemplo:

In [126]:
data = pd.DataFrame({
    'k1': ['one', 'two'] * 3 + ['two'],
    'k2': [1,1,2,3,3,4,4]
})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


O método *duplicated* de DataFrame devolve uma Series booleana
informando se cada linha é uma duplicata (foi observada em uma
linha anterior) ou não:

In [127]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

Relacionado a esse caso, temos *drop_duplicates* , que devolve um
DataFrame com dados em que o array *duplicated* é *False*:

In [128]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Por padrão, esses dois métodos consideram todas as colunas; de
forma alternativa, podemos especificar qualquer subconjunto delas
na detecção de duplicatas. Suponha que tivéssemos uma coluna
adicional de valores e quiséssemos filtrar as duplicatas somente
com base na coluna *'k1'*:

In [129]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [130]:
data.drop_duplicates(subset=['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


*duplicated* e *drop_duplicates*, por padrão, mantêm a primeira
combinação de valores observados. Passar *keep='last'* devolverá a
última:

In [131]:
data.drop_duplicates(subset=['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Transformando dados usando uma função ou um mapeamento
Para muitos conjuntos de dados, talvez você queira fazer algumas
transformações com base nos valores de um array, uma Series ou
uma coluna de um DataFrame. Considere os seguintes dados
hipotéticos coletados acerca de vários tipos de carnes:

In [132]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                    'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Suponha que quiséssemos adicionar uma coluna informando o tipo
do animal do qual cada alimento é proveniente. Vamos criar um
mapeamento entre cada tipo distinto de carne e o tipo do animal:

In [133]:
meat_to_animal = {
'bacon': 'pig',
'pulled pork': 'pig',
'pastrami': 'cow',
'corned beef': 'cow',
'honey ham': 'pig',
'nova lox': 'salmon'
}

In [134]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [135]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


Poderíamos também ter passado uma função que fizesse todo o
trabalho:

In [136]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### Substituindo valores
Preencher dados ausentes com o método *fillna* é um caso especial
da substituição mais genérica de valores. Conforme já vimos, *map*
pode ser usado para modificar um subconjunto de valores em um
objeto, porém *replace* oferece uma forma mais simples e mais flexível
de fazer isso. Considere a Series a seguir:

In [137]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

Os valores *-999* podem ser valores de sentinela para dados
ausentes. Para substituí-los por valores NA, compreensíveis pelo
pandas, podemos usar *replace*, gerando uma nova Series (a menos
que você passe *inplace=True*):

In [138]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

Se você quiser substituir diversos valores de uma só vez, passe
uma lista e, em seguida, o valor para substituição:

In [139]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

Para usar um substituto diferente para cada valor, passe uma lista
deles:

In [140]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

O argumento especificado também pode ser um dicionário:

In [141]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

OBS: *O método data.replace é diferente de data.str.replace, que faz uma
substituição de string em todos os elementos. Veremos esses métodos de
string em Series mais adiante neste capítulo.*

### Renomeando os índices dos eixos
Assim como os valores em uma Series, os rótulos dos eixos podem
ser transformados de modo semelhante por uma função ou alguma
forma de mapeamento, a fim de gerar objetos novos com rótulos
diferentes. Também podemos modificar os eixos in-place, sem criar
uma nova estrutura de dados. Eis um exemplo simples:

In [142]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four']
                    )
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


Como em uma Series, os índices de um eixo têm um método *map*:

In [143]:
transform = lambda x: x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

Podemos fazer uma atribuição para index, modificando o DataFrame in-place:

In [144]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


Se você quiser criar uma versão transformada de um conjunto de
dados sem modificar os dados originais, um método útil será *rename*:

In [145]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


Observe que *rename* pode ser usado em conjunto com um objeto do
tipo dicionário fornecendo novos valores para um subconjunto dos
rótulos de um eixo:

In [146]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


*rename* evita que você tenha o trabalho de copiar o DataFrame
manualmente e definir seus atributos *index* e *columns*. Caso você
queira modificar um conjunto de dados in-place, passe *inplace=True*:

In [147]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### Discretização e compartimentalização (binning)
Dados contínuos com frequência são discretizados ou, de modo
alternativo, separados em “compartimentos” (bins) para análise.
Suponha que tenhamos dados sobre um grupo de pessoas em um
estudo e queremos agrupá-las em conjuntos de idades discretas:

In [148]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Vamos dividir esses dados em compartimentos de 18 a 25, 26 a 35,
36 a 60 e, por fim, 61 anos ou mais. Para isso, utilize *cut*, uma
função do pandas:

In [149]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

O objeto devolvido pelo pandas é um objeto *Categorical* especial. A
saída que vemos descreve os compartimentos calculados pelo
*pandas.cut*. Podemos tratá-la como um array de strings informando o
nome do compartimento; internamente, ela contém um array
*categories* que especifica os nomes distintos das categorias, junto
com rótulos para os dados de *ages* no atributo *codes*:

In [150]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [151]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [152]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

Observe que *pd.value_counts(cats)* são os contadores de
compartimentos para o resultado de *pandas.cut*.

De forma consistente com a notação matemática para intervalos, um
parêntese significa que o lado está *aberto*, enquanto o colchete
indica que está *fechado* (é inclusivo). Podemos alterar o lado que
está fechado passando *right=False*:

In [153]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

Também podemos especificar nossos próprios nomes de
compartimentos passando uma lista ou um array para a opção *labels*:

In [154]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

Se passarmos um número inteiro de compartimentos para *cut*, em
vez de passar fronteiras explícitas, ele calculará compartimentos de
tamanhos iguais com base nos valores mínimo e máximo dos
dados. Considere o caso de alguns dados uniformemente
distribuídos, divididos em quartos:

In [155]:
data = np.random.rand(20)

In [156]:
pd.cut(data, 4, precision=2)

[(0.79, 0.97], (0.6, 0.79], (0.79, 0.97], (0.41, 0.6], (0.22, 0.41], ..., (0.6, 0.79], (0.22, 0.41], (0.41, 0.6], (0.22, 0.41], (0.22, 0.41]]
Length: 20
Categories (4, interval[float64]): [(0.22, 0.41] < (0.41, 0.6] < (0.6, 0.79] < (0.79, 0.97]]

A opção *precision=2* limita a precisão decimal em dois dígitos.

Uma função intimamente relacionada, *qcut*, compartimenta os dados
com base nos quantis da amostra. Conforme a distribuição dos
dados, usar *cut* em geral não resultará em cada compartimento com
o mesmo número de pontos de dados. Como *qcut* utiliza quantis da
amostra, por definição, você obterá compartimentos grosseiramente
de mesmo tamanho:

In [157]:
data = np.random.randn(1000) # Normalmente Distribuidos

In [158]:
cats = pd.qcut(data, 4) # Separa em quantis
cats

[(0.62, 3.928], (-2.9499999999999997, -0.684], (0.62, 3.928], (0.62, 3.928], (0.62, 3.928], ..., (-2.9499999999999997, -0.684], (0.62, 3.928], (-0.0265, 0.62], (-0.0265, 0.62], (-2.9499999999999997, -0.684]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0265] < (-0.0265, 0.62] <
                                    (0.62, 3.928]]

In [159]:
pd.value_counts(cats)

(0.62, 3.928]                    250
(-0.0265, 0.62]                  250
(-0.684, -0.0265]                250
(-2.9499999999999997, -0.684]    250
dtype: int64

De modo semelhante a *cut*, podemos passar nossos próprios quantis
(números entre 0 e 1, inclusive):

In [160]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(1.287, 3.928], (-2.9499999999999997, -1.187], (-0.0265, 1.287], (-0.0265, 1.287], (-0.0265, 1.287], ..., (-2.9499999999999997, -1.187], (-0.0265, 1.287], (-0.0265, 1.287], (-0.0265, 1.287], (-1.187, -0.0265]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -1.187] < (-1.187, -0.0265] < (-0.0265, 1.287] <
                                    (1.287, 3.928]]

Voltaremos a discutir *cut* e *qcut* mais adiante neste capítulo, durante
a nossa discussão sobre agregação e operações de grupo, pois
essas funções de discretização são particularmente úteis para
análise de quantis e de grupos.

### Detectando e filtrando valores discrepantes
Filtrar ou transformar valores discrepantes (outliers) é, em boa
medida, uma questão de aplicar operações de array. Considere um
DataFrame com alguns dados normalmente distribuídos:

In [161]:
data = pd.DataFrame(np.random.randn(1000, 4))

In [162]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.045383,0.024079,-0.004794,-0.050389
std,0.997796,1.008737,0.996599,0.996706
min,-3.645860,-3.184377,-3.745356,-3.428254
25%,-0.600254,-0.612162,-0.697084,-0.747478
50%,0.040150,-0.013609,-0.029924,-0.088274
75%,0.746527,0.690847,0.699046,0.623331
max,2.653656,3.525865,2.735527,3.366626


Suponha que quiséssemos encontrar os valores que excedessem 3
em valor absoluto em uma das colunas:

In [163]:
col = data[2]

In [164]:
col[np.abs(col) > 3]

38    -3.399312
133   -3.745356
Name: 2, dtype: float64

Para selecionar todas as linhas que tenham um valor que exceda 3
ou –3, podemos utilizar o método *any* em um DataFrame booleano:

In [165]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
38,0.457246,-0.025907,-3.399312,-0.974657
57,1.951312,3.260383,0.963301,1.201206
133,0.508391,-0.196713,-3.745356,-1.520113
232,-0.242459,-3.056990,1.918403,-0.578828
255,0.682841,0.326045,0.425384,-3.428254
319,1.179227,-3.184377,1.369891,-1.074833
541,-3.548824,1.553205,-2.186301,1.277104
632,-0.578093,0.193299,1.397822,3.366626
779,-0.207434,3.525865,0.283070,0.544635
800,-3.645860,0.255475,-0.549574,-1.907459


Valores podem ser definidos com base nesses critérios. Eis um
código para eliminar os valores que estejam fora do intervalo de –3
a 3:

In [166]:
data[np.abs(data) > 3] = np.sign(data) * 3

In [167]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.046577,0.023534,-0.003650,-0.050328
std,0.993777,1.005496,0.992788,0.994152
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.600254,-0.612162,-0.697084,-0.747478
50%,0.040150,-0.013609,-0.029924,-0.088274
75%,0.746527,0.690847,0.699046,0.623331
max,2.653656,3.000000,2.735527,3.000000


A instrução *np.sign(data)* gera valores 1 e –1 com base no fato de os
valores em data serem positivos ou negativos:

In [168]:
np.sign(data).head()

,0,1,2,3
0,-1.0,-1.0,1.0,-1.0
1,-1.0,1.0,-1.0,-1.0
2,1.0,-1.0,1.0,-1.0
3,1.0,-1.0,1.0,-1.0
4,-1.0,-1.0,-1.0,-1.0


### Permutação e amostragem aleatória
Permutar (reordenar aleatoriamente) uma Series ou as linhas de um
DataFrame é fácil utilizando a função *numpy.random.permutation*.
Chamar *permutation* com o tamanho do eixo que você quer permutar
gera um array de inteiros informando a nova ordem:

In [169]:
df = pd.DataFrame(np.arange(5 * 4).reshape(5, 4))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [170]:
sampler = np.random.permutation(5)
sampler

array([2, 4, 1, 0, 3])

Esse array pode então ser usado na indexação baseada em *iloc* ou
na função *take* equivalente:

In [171]:
df.take(sampler)

,0,1,2,3
2,8,9,10,11
4,16,17,18,19
1,4,5,6,7
0,0,1,2,3
3,12,13,14,15


Para selecionar um subconjunto aleatório sem substituição, o
método *sample* pode ser usado em Series e em DataFrame:

In [172]:
df.sample(n=3)

,0,1,2,3
2,8,9,10,11
4,16,17,18,19
1,4,5,6,7


Para gerar uma amostra com substituição (a fim de permitir opções
repetidas), passe *replace=True* para *sample*:

In [173]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

4    4
1    7
2   -1
3    6
4    4
1    7
4    4
0    5
3    6
0    5
dtype: int64

### Calculando variáveis indicadoras/dummy

Outro tipo de transformação para modelagem estatística ou
aplicações de aprendizado de máquina (machine learning) consiste
em converter uma variável de categorias em uma matriz *“dummy”*
ou *“indicadora”*. Se uma coluna em um DataFrame tiver *k* valores
distintos, poderíamos derivar uma matriz ou um DataFrame com *k*
colunas contendo somente 1s e 0s. O pandas tem uma função
*get_dummies* para isso, embora criar uma função por conta própria
não seria difícil. Vamos retomar um exemplo anterior com DataFrame:

In [174]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)
                   })
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


Em alguns casos, talvez você queira adicionar um prefixo às
colunas no DataFrame indicador, que poderá então ser mesclado
com os outros dados. *get_dummies* tem um argumento de prefixo para
isso:

In [175]:
dummies = pd.get_dummies(df['key'], prefix='key')
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [176]:
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


Se uma linha de um DataFrame pertencer a várias categorias, a
situação se torna um pouco mais complicada.


In [177]:
path = '/content/drive/MyDrive/Análise de Dados/CSVs e TXTs/movies.csv'
movies = pd.read_csv(path, delimiter=';')
movies

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


Adicionar variáveis indicadoras para cada gênero exige um pouco
de manipulação nos dados. Inicialmente extraímos a lista de
gêneros únicos do conjunto de dados:

In [178]:
all_genres = []
for i in movies.genres:
  # all_genres.append(i.split('|'))
  all_genres.extend(i.split('|'))

all_genres

['Animation',
 "Children's",
 'Comedy',
 'Adventure',
 "Children's",
 'Fantasy',
 'Comedy',
 'Romance',
 'Comedy',
 'Drama',
 'Comedy',
 'Action',
 'Crime',
 'Thriller',
 'Comedy',
 'Romance',
 'Adventure',
 "Children's",
 'Action',
 'Action',
 'Adventure',
 'Thriller']

In [179]:
genres = pd.unique(all_genres)
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller'], dtype=object)

Uma maneira de construir o DataFrame indicador é começar com
um DataFrame contendo apenas zeros:

In [180]:
zeros_matrix = np.zeros((len(movies), len(genres)))
zeros_matrix

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [181]:
dummies = pd.DataFrame(zeros_matrix, columns=genres)
dummies

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Agora itere pelos filmes e defina entradas para cada linha de *dummies* com 1. Para isso, utilize *dummies.columns* a fim de calcular os
índices das colunas para cada gênero:

In [182]:
gen = movies.genres[0]
gen.split('|')

['Animation', "Children's", 'Comedy']

In [183]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2])

Então podemos usar *.iloc* para definir valores com base nesses
índices:

In [184]:
for i, gen in enumerate(movies.genres):
  indices = dummies.columns.get_indexer(gen.split('|'))
  dummies.iloc[i, indices] = 1

Em seguida, como fizemos antes, podemos combinar esses dados
com *movies*:

In [185]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]

movie_id                                      1
title                          Toy Story (1995)
genres              Animation|Children's|Comedy
Genre_Animation                               1
Genre_Children's                              1
Genre_Comedy                                  1
Genre_Adventure                               0
Genre_Fantasy                                 0
Genre_Romance                                 0
Genre_Drama                                   0
Genre_Action                                  0
Genre_Crime                                   0
Genre_Thriller                                0
Name: 0, dtype: object

In [186]:
movies_windic

,movie_id,title,genres,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Adventure,Genre_Fantasy,Genre_Romance,Genre_Drama,Genre_Action,Genre_Crime,Genre_Thriller
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,Heat (1995),Action|Crime|Thriller,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
6,7,Sabrina (1995),Comedy|Romance,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,8,Tom and Huck (1995),Adventure|Children's,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8,9,Sudden Death (1995),Action,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,10,GoldenEye (1995),Action|Adventure|Thriller,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0


OBS: *Para dados bem maiores, esse método de construir variáveis indicadoras
com vários membros não é particularmente ágil. Seria melhor escrever uma
função de nível mais baixo que escrevesse diretamente em um array NumPy
e então encapsular o resultado em um DataFrame.*

Uma receita útil para aplicações estatísticas é combinar *get_dummies*
com uma função de discretização como *cut*:

In [187]:
np.random.seed(12345)
values = np.random.rand(10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [188]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


### Manipulação de strings
Python tem sido uma linguagem popular para manipulação de dados
brutos há muito tempo, em parte por causa de sua facilidade de uso
para processamento de strings e de texto. A maior parte das
operações em texto são simplificadas com os métodos embutidos
do objeto string. Para uma correspondência de padrões e
manipulações de texto mais complexas, o uso de expressões
regulares talvez seja necessário. O pandas complementa a mistura
permitindo aplicar strings e expressões regulares de forma concisa
em arrays inteiros de dados, além de lidar com o transtorno dos
dados ausentes.

### Métodos de objetos string
Para muitas aplicações de manipulação de strings e de scripting, os
métodos embutidos de string são suficientes. Como exemplo, uma
string separada por vírgulas pode ser dividida em partes usando
*split*:

In [189]:
values = 'a,b, guido'
values.split(',')

['a', 'b', ' guido']

*Com frequência, split é usado em conjunto com strip para remover espaços em
branco (incluindo quebras de linha):*

In [190]:
pieces = [i.strip() for i in values.split(',')]
pieces

['a', 'b', 'guido']

Essas substrings poderiam ser concatenadas com um delimitador
de dois-pontos duplos usando a adição:

In [192]:
first, second, third = pieces
print(first + '::' + second + '::' + third)

a::b::guido


No entanto, esse não é um método genérico prático. Uma forma
mais rápida e pythônica consiste em passar uma lista ou uma tupla
para o método *join* na string '::':

In [193]:
'::'.join(pieces)

'a::b::guido'

Outros métodos dizem respeito à localização de substrings. Usar a
palavra reservada *in* de Python é a melhor maneira de detectar uma
substring, embora *index* e *find* também possam ser utilizados:

In [194]:
'guido' in values

True

In [195]:
values.index(',')

1

In [196]:
values.find(':')

-1

Observe que a diferença entre *find* e *index* é que *index* lança uma
exceção caso a string não seja encontrada (*versus* devolver –1):

Relacionado a esse caso, temos *count*, que devolve o número de
ocorrências de uma substring em particular:

In [197]:
values.count(',')

2

*replace* substituirá as ocorrências de um padrão por outro. É
comumente utilizado também para apagar padrões, passando uma
string vazia:

In [198]:
values.replace(',', '::')

'a::b:: guido'

In [199]:
values.replace(',', '')

'ab guido'

###Tabela – Métodos embutidos de string em Python
Argumento | Descrição
----------|-------------------
**count** | Devolve o número de ocorrências de uma substring na string, sem
|sobreposição.
**endswith** | Devolve True se a string terminar com o sufixo.
**startswith** | Devolve True se a string começar com o prefixo.
**join** | Utiliza a string como delimitadora para concatenar uma sequência
|de outras strings.
**index** | Devolve a posição do primeiro caractere de uma substring, se ela
|for encontrada em uma string: gera ValueError se não encontrar.
**find** | Devolve a posição do primeiro caractere da primeira ocorrência da
|substring na string; é como index, porém devolve –1 se não
|encontrar.
**rfind** | Devolve a posição do primeiro caractere da última ocorrência da
|substring na string; devolve –1 se não encontrar.
**replace** | Substitui ocorrências de uma string por outra string.
**strip, rstrip, lstrip** | Remove espaços em branco, incluindo quebras de   
|linha; é equivalente a x.strip() (e a rstrip e lstrip, respectivamente 
|para cada elemento.
**split** | Separa a string em uma lista de substrings usando o delimitador
|especificado.
**lower** | Converte os caracteres alfabéticos para letras minúsculas.
**upper** | Converte os caracteres alfabéticos para letras maiúsculas.
**casefold** | Converte os caracteres para letras minúsculas e converte quaisquer
|combinações variáveis de caracteres específicos de região para um
|formato comum comparável.
**ljust**, **rjust** | Justifica à esquerda ou à direita, respectivamente; preenche o lado
|oposto da string com espaços (ou com outro caractere de
|preenchimento) para devolver uma string com um tamanho mínimo.


### Expressões regulares
As *expressões regulares* oferecem uma maneira flexível para fazer
pesquisas ou correspondências (em geral, mais complexas) de
padrões de string em um texto. Uma única expressão, em geral
chamada de *regex*, é uma string composta de acordo com a
linguagem da expressão regular. O módulo embutido *re* de Python é
responsável pela aplicação de expressões regulares em strings;
apresentarei uma série de exemplos de seu uso nesta seção.

As funções do módulo *re* se enquadram em três categorias:
correspondência, substituição e separação de padrões.
Naturalmente, elas estão relacionadas; uma regex descreve um
padrão a ser localizado no texto, que pode então ser usado para
vários propósitos. Vamos analisar um exemplo simples: suponha
que quiséssemos separar uma string com um número variável de
caracteres de espaços em branco (tabulações, espaços e quebras
de linha). A regex que descreve um ou mais caracteres para
espaços em branco é *\s+*:

In [200]:
import re

In [204]:
text = 'foo bar\t baz \tqux'
print(text)

foo bar	 baz 	qux


In [202]:
re.split(r'\s+', text)

['foo', 'bar', 'baz', 'qux']

Quando chamamos *re.split('\s+', text)*, a expressão regular inicialmente
é *compilada*; então seu método *split* é chamado no texto que lhe é
passado. Podemos compilar a regex por conta própria com
*re.compile*, criando um objeto regex reutilizável:

In [206]:
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

Se, em vez disso, quiséssemos obter uma lista de todos os padrões
que correspondam à regex, o método *findall* poderia ser utilizado:

In [207]:
regex.findall(text)

[' ', '\t ', ' \t']

OBS: *Para evitar um escaping indesejado com \ em uma expressão regular,
utilize literais de string puros como r'C:\x' no lugar do 'C:\\x' equivalente.*

Criar um objeto regex com re.compile é altamente recomendado caso
você pretenda aplicar a mesma expressão a várias strings; fazer
isso lhe economizará ciclos de CPU.

*match* e *search* estão intimamente relacionados a *findall*. Enquanto
*findall* devolve todas as correspondências em uma string, *search*
devolve apenas a primeira. De modo mais rigoroso, *match* faz a
correspondência *somente* no início da string. Como um exemplo
menos trivial, vamos considerar um bloco de texto e uma expressão
regular capaz de identificar a maioria dos endereços de email:

In [208]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""

In [209]:
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+.[A-Z]{2,4}'

In [210]:
regex = re.compile(pattern, flags=re.IGNORECASE)

Usar *findall* no texto gera uma lista de endereços de email:

In [211]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

*search* devolve um objeto especial de correspondência para oprimeiro endereço de email no texto. Para a regex anterior, o objeto
de correspondência pode nos dizer apenas a posição de início e de
fim do padrão na string:

In [212]:
m = regex.search(text)
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [213]:
text[m.start():m.end()]

'dave@google.com'

*regex.match* devolve **None**, pois fará a correspondência somente se o
padrão ocorrer no início da string:

In [214]:
print(regex.match(text))

None


Relacionado a esse caso, temos sub, que devolverá uma nova string
com as ocorrências do padrão substituídas por uma nova string:

In [215]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



Suponha que quiséssemos encontrar os endereços de email e,
simultaneamente, segmentar cada endereço em seus três
componentes: nome do usuário, nome do domínio e sufixo do
domínio. Para isso, coloque parênteses em torno das partes do
padrão a fim de segmentá-lo:

In [216]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'

In [217]:
regex = re.compile(pattern=pattern, flags=re.IGNORECASE)

Um objeto de correspondência gerado por essa regex modificada
devolve uma tupla dos componentes do padrão com seu método
groups:

In [219]:
m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

*findall* devolve uma lista de tuplas se o padrão tiver grupos:

In [220]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

*sub* também tem acesso aos grupos em cada correspondência,
usando símbolos especiais como \1 e \2 . O símbolo \1 corresponde
ao primeiro grupo identificado, \2 corresponde ao segundo grupo, e
assim sucessivamente:

In [222]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



### Tabela – Métodos de expressões regulares

Argumento | Descrição
----------|-------------
**findall** | Devolve todos os padrões correspondentes em uma string, sem
|sobreposição, na forma de uma lista
**finditer** | É como findall, porém devolve um iterador
**match** | Corresponde padrões no início da string e, opcionalmente,
|segmenta componentes do padrão em grupos; se houver uma
|correspondência com o padrão, devolve um objeto de
|correspondência; caso contrário, devolve None
**search** | Pesquisa a string para verificar se há uma correspondência com o
|padrão; em caso afirmativo, devolve um objeto de correspondência.
|De modo diferente de match, a correspondência pode se dar em
|qualquer ponto da string, em oposição a ocorrer somente no início
**split** | Separa a string em partes a cada ocorrência do padrão
**sub, subn** | Substitui todas (sub) ou as n primeiras (subn) ocorrências do
|padrão em uma string por uma expressão substituta; utiliza os
|símbolos \1, \2, ... para referenciar os elementos de grupo da
|correspondência na string de substituição

### Funções de string vetorizadas no pandas
Limpar um conjunto de dados desorganizado para análise em geral
exige muita manipulação e regularização de strings. Para complicar
mais ainda a situação, uma coluna contendo strings ocasionalmente
terá dados ausentes:

In [223]:
data = {'Dave': 'dave@google.com', 
        'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 
        'Wes': np.nan}

In [224]:
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [225]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

Podemos aplicar métodos de string e de expressões regulares
(passando uma *lambda* ou outra função) para cada valor usando
*data.map*, mas haverá uma falha em valores NA (nulos). Para lidar
com isso, Series tem métodos orientados a arrays para operações
em string, que ignoram valores NA. Eles são acessados por meio do
atributo *str* de Series; por exemplo, poderíamos verificar se cadaendereço de email contém '*gmail*' usando *str.contains*:

In [226]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

Expressões regulares também podem ser usadas, junto com
qualquer opção de *re*, como IGNORECASE:

In [227]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [228]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

Há duas maneiras para a obtenção de elementos vetorizados. Você
pode usar *str.get* ou indexar no atributo *str*:

In [229]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

Para acessar os elementos nas listas embutidas, podemos passar
um índice para qualquer uma dessas funções:

In [237]:
data.str[: 5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

### Tabela – Listagem parcial dos métodos de string vetorizados
Método | Descrição
-------|-------------------
**cat** | Concatena strings em todos os elementos, com um delimitador
|opcional
**contains** | Devolve um array booleano se cada string contiver um padrão/uma regex
**count** | Conta as ocorrências do padrão
**extract** | Utiliza uma expressão regular com grupos para extrair uma ou mais
|strings de uma Series de strings; o resultado será um DataFrame com
|uma coluna por grupo
**endswith** | Equivalente a x.endswith(pattern) para cada elemento
**startswith** | Equivalente a x.startswith(pattern) para cada elemento
**findall** | Calcula uma lista com todas as ocorrências de um padrão/uma regex
|para cada string
**get** | Indexa cada elemento (obtém o i-ésimo elemento)
**isalnum** | Equivalente ao str.alnum embutido
**isalpha** | Equivalente ao str.isalpha embutido
**isdecimal** | Equivalente ao str.isdecimal embutido
**isdigit** | Equivalente ao str.isdigit embutido
**islower** | Equivalente ao str.islower embutido
**isnumeric** | Equivalente ao str.isnumeric embutido
**isupper** | Equivalente ao str.isupper embutido
**join** | Junta strings em cada elemento da Series utilizando o separador
|especificado
**len** | Calcula o tamanho de cada string
**lower, upper** | Converte para letras minúsculas ou maiúsculas; equivalente a
|x.lower() ou a x.upper() para cada elemento
**match** | Usa re.match com a expressão regular especificada em cada
|elemento, devolvendo os grupos com os quais houve uma
|correspondência, na forma de uma lista
**pad** | Adiciona espaços em branco à esquerda, à direita ou nos dois lados
|das strings
**center** | Equivalente a pad(side='both')
**repeat** | Duplica valores (por exemplo, s.str.repeat(3) é equivalente a x * 3 para
cada string)
**replace** | Substitui ocorrências do padrão/da regex por outra string
**slice** | Fatia cada string da Series
**split** | Separa as string no delimitador ou na expressão regular
**strip** | Remove espaços em branco de ambos os lados, incluindo quebras de
|linha
**rstrip** | Remove espaços em branco do lado direito
**lstrip** | Remove espaços em branco do lado esquerdo